Импорт библиотек

In [ ]:
import gdown
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch
from tqdm import tqdm
from google.colab import files
import os

Загрузка файлов модели

In [ ]:
file_id = '1evL7ZWaS41fSZzzhSnbjQjUb5itUlgF3'
config_id = '1cqvSKNRfcXxGzzs1ecPff6RUdIqgdGkK'
special_tokens = '1wRrlXOEm6prK-Fn8pACaVTuIdM508YgG'
token_config = '1Vs7QnboBKlL0tTAgFq4FVPh8iaTVwOuA'
vocab_path = '1eMf0hdAtmwn2KsJRyJ64_qtN8rz0zhhm'

gdown.download(f'https://drive.google.com/uc?id={file_id}', 'model.safetensors', quiet=False)
gdown.download(f'https://drive.google.com/uc?id={config_id}', 'config.json', quiet=False)
gdown.download(f'https://drive.google.com/uc?id={special_tokens}', 'special_tokens_map.json', quiet=False)
gdown.download(f'https://drive.google.com/uc?id={token_config}', 'tokenizer_config.json', quiet=False)
gdown.download(f'https://drive.google.com/uc?id={vocab_path}', 'vocab.txt', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1evL7ZWaS41fSZzzhSnbjQjUb5itUlgF3
From (redirected): https://drive.google.com/uc?id=1evL7ZWaS41fSZzzhSnbjQjUb5itUlgF3&confirm=t&uuid=24ee5ede-231d-406d-876c-2402be1b7e1c
To: /content/model.safetensors
100%|██████████| 712M/712M [00:08<00:00, 79.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1cqvSKNRfcXxGzzs1ecPff6RUdIqgdGkK
To: /content/config.json
100%|██████████| 6.49k/6.49k [00:00<00:00, 8.81MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wRrlXOEm6prK-Fn8pACaVTuIdM508YgG
To: /content/special_tokens_map.json
100%|██████████| 125/125 [00:00<00:00, 393kB/s]
Downloading...
From: https://drive.google.com/uc?id=1Vs7QnboBKlL0tTAgFq4FVPh8iaTVwOuA
To: /content/tokenizer_config.json
100%|██████████| 1.30k/1.30k [00:00<00:00, 3.01MB/s]
Downloading...
From: https://drive.google.com/uc?id=1eMf0hdAtmwn2KsJRyJ64_qtN8rz0zhhm
To: /content/vocab.txt
100%|██████████| 1.65M/1.65M [00:00<00:00, 48.8MB/s]


'vocab.txt'

In [ ]:
# Фиксированные параметры
MAX_LENGTH = 150
SCALING_FACTOR = 0.7  # Порог = максимальная вероятность * 0.7

def check_model_files():
    required_files = ['config.json', 'vocab.txt', 'model.safetensors', 'tokenizer_config.json', 'special_tokens_map.json']
    missing_files = [f for f in required_files if not os.path.exists(f)]
    if missing_files:
        raise FileNotFoundError(f"Отсутствуют файлы модели: {', '.join(missing_files)}")
    print("✅ Все файлы модели найдены")

class PredictionDataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        if not isinstance(text, str):
            text = str(text)

        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=MAX_LENGTH,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
        }

def load_model_and_tokenizer():
    try:
        check_model_files()
        tokenizer = BertTokenizer.from_pretrained(".")
        model = BertForSequenceClassification.from_pretrained(".")
        return tokenizer, model
    except Exception as e:
        print(f"⚠️ Ошибка: {str(e)}. Убедитесь, что все файлы модели загружены в текущую папку.")
        exit(1)

def main():
    print("Добро пожаловать в систему предсказаний!")

    # Загрузка модели
    tokenizer, model = load_model_and_tokenizer()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    # Загрузка CSV через браузер без заголовка
    print("\nЗагрузите CSV-файл:")
    uploaded = files.upload()
    if not uploaded:
        print("⚠️ CSV-файл не загружен")
        return

    csv_file = list(uploaded.keys())[0]
    # Читаем CSV без заголовка
    data = pd.read_csv(csv_file, header=None)  # header=None: игнорируем заголовок

    # Берем первый столбец как тексты
    texts = data.iloc[:, 0].values  # Все строки, первый столбец

    # Инференс
    dataset = PredictionDataset(texts=texts, tokenizer=tokenizer)
    dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

    all_active_labels = []

    for batch in tqdm(dataloader, desc="Предсказание"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        with torch.no_grad():
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            probs_batch = torch.sigmoid(outputs.logits).cpu().numpy()
            probs_batch = probs_batch.squeeze(1) if probs_batch.ndim == 3 else probs_batch

            # Адаптивный порог для каждого примера
            adaptive_thresholds = np.max(probs_batch, axis=1) * SCALING_FACTOR
            active_labels_batch = (probs_batch > adaptive_thresholds[:, np.newaxis]).astype(int)
            # Собираем индексы меток (+1 для сдвига)
            active_indices_batch = [
                ",".join(map(str, (np.where(active_labels)[0] + 1)))
                if np.any(active_labels) else ""
                for active_labels in active_labels_batch
            ]

        all_active_labels.extend(active_indices_batch)

    # Создаем DataFrame с названием столбцов вручную
    output_df = pd.DataFrame({
        "text": texts,
        "predicted_labels": all_active_labels
    })

    output_path = "predictions.csv"
    output_df.to_csv(output_path, index=False)
    # Вывод первых 10 строк
    print(output_df.head(10))
    files.download(output_path)
    print(f"✅ Результаты сохранены в {output_path} и доступны для скачивания")

if __name__ == "__main__":
    main()

Добро пожаловать в систему предсказаний!
✅ Все файлы модели найдены

Загрузите CSV-файл:


Saving transformer_tests.csv to transformer_tests (1).csv


Предсказание: 100%|██████████| 1/1 [00:04<00:00,  4.38s/it]

                                                text predicted_labels
0  Итак, предметы выбраны. ВУЗы тоже. А дальше са...           75,131
1  Начну с самого начала. В ноябре 2020 года в ра...              5,6
2  Здравствуйте всем! Хочу услышать ваше мнение п...         27,31,36
3  Когда же экзамен совсем не удавалось сдать, пр...         72,84,86
4  Интересует стипендия на бюджете или кто скольк...               81
5  В хороших вузах низкая стипендия, им привлекат...            67,79
6  Проработала 15 лет в университете (Кемеровский...         65,79,81
7  Зп в вузах понятие растяжимое. Есть на контрак...            79,81
8  – Капитального ремонта там не было больше 75 л...            24,30
9  Всем привет, сдавала химию, русский, базовую м...            27,36


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Результаты сохранены в predictions.csv и доступны для скачивания


In [ ]:
df = pd.read_csv("predictions.csv")
df

,text,predicted_labels
0,"Итак, предметы выбраны. ВУЗы тоже. А дальше са...","75,131"
1,Начну с самого начала. В ноябре 2020 года в ра...,"5,6"
2,Здравствуйте всем! Хочу услышать ваше мнение п...,"27,31,36"
3,"Когда же экзамен совсем не удавалось сдать, пр...","72,84,86"
4,Интересует стипендия на бюджете или кто скольк...,81
5,"В хороших вузах низкая стипендия, им привлекат...","67,79"
6,Проработала 15 лет в университете (Кемеровский...,"65,79,81"
7,Зп в вузах понятие растяжимое. Есть на контрак...,"79,81"
8,– Капитального ремонта там не было больше 75 л...,"24,30"
9,"Всем привет, сдавала химию, русский, базовую м...","27,36"


No charts were generated by quickchart


In [ ]:
labels_df = pd.read_excel("/content/Классификации.xlsx")
labels_df

,id,level_3
0,1,Состояние зданий и помещений дошкольных учрежд...
1,2,Домашнее обучение (дошкольники)
2,3,Финансовая доступность дошкольных учреждений
3,4,Наличие/Отсутствие дошкольных учреждений
4,5,Наличие мест в дошкольных учреждениях
...,...,...
127,128,Безопасность в учреждениях дополнительного про...
128,129,Прием/поступление в учреждения дополнительного...
129,130,Прием/поступление в вузы по льготе
130,131,Прием/поступление в вузы победителе и призеров...


In [ ]:
# Создаём функцию для сопоставления нескольких меток
def map_labels(label_str, label_map):
    try:
        ids = [int(x.strip()) for x in label_str.split(",")]
        return ", ".join([label_map[id] for id in ids if id in label_map])
    except:
        return ""

# Создаём словарь {id: level_3} для быстрого поиска
label_dict = dict(zip(labels_df['id'], labels_df['level_3']))

# Применяем к каждой строке
df['level_3'] = df['predicted_labels'].apply(lambda x: map_labels(x, label_dict))

# Сохраняем результат
df[['text', 'level_3']].to_csv("predictions_with_level3.csv", index=False)

df[['text', 'level_3']]

,text,level_3
0,"Итак, предметы выбраны. ВУЗы тоже. А дальше са...","Прием/поступление в вузы на целевое обучение, ..."
1,Начну с самого начала. В ноябре 2020 года в ра...,"Наличие мест в дошкольных учреждениях, Дошколь..."
2,Здравствуйте всем! Хочу услышать ваше мнение п...,"Качество образования в школах, Цифровизация об..."
3,"Когда же экзамен совсем не удавалось сдать, пр...","Конфликтные ситуации в вузах, Сбор денег, корр..."
4,Интересует стипендия на бюджете или кто скольк...,Оплата труда и меры социальной поддержки сотру...
5,"В хороших вузах низкая стипендия, им привлекат...",Наличие бюджетных мест в высших учебных заведе...
6,Проработала 15 лет в университете (Кемеровский...,Оплата труда и меры социальной поддержки сотру...
7,Зп в вузах понятие растяжимое. Есть на контрак...,"Кадровые вопросы, трудоустройство в учреждения..."
8,– Капитального ремонта там не было больше 75 л...,"Состояние зданий и помещений школ, Условия про..."
9,"Всем привет, сдавала химию, русский, базовую м...","Качество образования в школах, Экзамены в школе"
